In [4]:
import numpy as np
import pandas as pd


id_stopword_dict = pd.read_csv('/Users/imamghozali/Documents/Master/Semester1/RPL/Data/stopwordbahasa.csv', header=None)
id_stopword_dict = id_stopword_dict.rename(columns={0: 'stopword'})

# print(id_stopword_dict)
data = pd.read_csv(
    "/Users/imamghozali/Documents/Master/Semester1/RPL/Data/re_dataset.csv", error_bad_lines=False, encoding='latin-1')
data1 = data.drop(['Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion', 'HS_Race',
                  'HS_Physical', 'HS_Gender', 'HS_Other', 'HS_Weak', 'HS_Moderate', 'HS_Strong'], axis=1)

alay_dict = pd.read_csv('/Users/imamghozali/Documents/Master/Semester1/RPL/Data/new_kamusalay.csv', encoding='latin-1', header=None)
alay_dict = alay_dict.rename(columns={0: 'original', 
                                      1: 'replacement'})
# print(data1.values[0][0])

ModuleNotFoundError: No module named 'pandas'

In [ ]:
import seaborn as sns

sns.countplot('HS',data=data1)


In [ ]:
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
from nltk.tokenize import word_tokenize 
import nltk
# from keras.preprocessing.text import Tokenizer

# nltk.download()

def lowercase(text):
    return text.lower()


def remove_unnecessary_char(text):
    # Remove every URL
    text = re.sub(
        '((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', ' ', text)
    text = re.sub('\n', ' ', text)  # Remove every '\n'
    text = re.sub('\r', ' ', text)  # Remove every '\r'
    text = re.sub('(?i)rt', ' ', text)  # Remove every retweet symbol
    text = re.sub('@[^\s]+[ \t]', '', text)  # Remove every username
    text = re.sub('(?i)user', '', text)  # Remove every username
    text = re.sub('(?i)url', ' ', text)  # Remove every url
    text = re.sub(r'\\x..', ' ', text)  # Remove every emoji
    text = re.sub('  +', ' ', text)  # Remove extra spaces
    # Remove characters repeating more than twice
    text = re.sub(r'(\w)\1{2,}', r'\1\1', text)

    return text


def remove_nonaplhanumeric(text):
    text = re.sub('[^0-9a-zA-Z]+', ' ', text)
    return text


alay_dict_map = dict(zip(alay_dict['original'], alay_dict['replacement']))


def normalize_alay(text):
    return ' '.join([alay_dict_map[word] if word in alay_dict_map else word for word in text.split(' ')])


def remove_stopword(text):
    text = ' '.join(
        ['' if word in id_stopword_dict.stopword.values else word for word in text.split(' ')])
    text = re.sub('  +', ' ', text)  # Remove extra spaces
    text = text.strip()
    return text


def stemming(text):
    return stemmer.stem(text)


def preprocess(text):
    text = lowercase(text)
    text = remove_unnecessary_char(text)
    text = remove_nonaplhanumeric(text)
    text = normalize_alay(text)
    text = remove_stopword(text)
    text = stemming(text)
    text = nltk.tokenize.word_tokenize(text)
    ordered_tokens = set()
    result = []
    # filter duplicate words
    # for word in text:
    #     if word not in ordered_tokens:
    #         ordered_tokens.add(word)
    #         result.append(word)
    # text = result      
    return text

print(preprocess("@saidaqil Andi kerap melakukan transaksi rutin secara daring atau online. Menurut Andi belanja online lebih praktis & murah."))
print(preprocess("@saidaqil apa maksud pernyataan. Anda? Apa anda mengaminkan kriminalisasi Ahok?? Hati2 jg dg mulutmu pak!"))
# data1 = data1.drop(['HS'], axis=1)
# result = []
# for item in data1.iloc[:, 0]:
#     # for num in item:    
#         # print(preprocess(item))
#         result.append(preprocess(item))

# print(result[0])

In [ ]:
x=data1['Tweet']
y=data1['HS']

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42,test_size=0.2)
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))

In [ ]:
import string

# Create an iterator object that returns words properly formatted for Word2Vec training
class TweetIterator:
    def __init__(self, dataset):
        self.dataset = dataset

        # The text contains a stupid special character, hence the + '–'
        self.translator = str.maketrans('', '', string.punctuation + '–')

    def __iter__(self):
        for tweet in self.dataset:
            # Make all characters lower-case
            tweet = tweet.lower()
            sentence= preprocess(tweet)
            words = [w for w in sentence if w != '']
            # for sentence in tweet.split('.')[:-1]:
            #     # Clean string of all punctuation
            #     # sentence = sentence.translate(self.translator)
            #     sentence= preprocess(sentence)
            #     words = [w for w in sentence]

            yield words

sentences = TweetIterator(x_train)
# print(list(sentences))


In [ ]:
import gensim

w2v_model = gensim.models.Word2Vec(sentences, window=2, min_count=10, workers=8)

# Retrieve the weights from the model. This is used for initializing the weights
# in a Keras Embedding layer later
w2v_weights = w2v_model.wv.vectors
vocab_size, embedding_size = w2v_weights.shape

print("Vocabulary Size: {} - Embedding Dim: {}".format(vocab_size, embedding_size))



In [ ]:
print(w2v_model.wv.most_similar('anjir', topn=3))

def word2token(word):
    try:
        return w2v_model.wv.key_to_index[word]
    # If word is not in index return 0. I realize this means that this
    # is the same as the word of index 0 (i.e. most frequent word), but 0s
    # will be padded later anyway by the embedding layer (which also
    # seems dirty but I couldn't find a better solution right now)
    except KeyError:
        return 0
def token2word(token):
    return w2v_model.wv.index2word[token]

In [ ]:
import matplotlib.pyplot as plt

categories, ccount = np.unique(y_train, return_counts=True)

# Using a figsize this big seems dirty. It's the only way I figured to make
# the x labels not overlap. I suck at plotting, sorry
plt.figure(figsize=(15, 8))
plt.title("Dataset Category Distribution")
plt.bar(categories, ccount, align='center')
plt.show()

In [ ]:
# Import everything that will be used
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing.sequence import pad_sequences
import string
# Create an iterator that formats data from the dataset proper for
# LSTM training

# Sequences will be padded or truncated to this length
MAX_SEQUENCE_LENGTH = 200

# Samples of categories with less than this number of samples will be ignored
DROP_THRESHOLD = 100

class SequenceIterator:
    def __init__(self, dataset_x,dataset_y, drop_threshold, seq_length):
        self.dataset_x = dataset_x
        self.dataset_y = dataset_y
        self.translator = str.maketrans('', '', string.punctuation + '–')
        self.categories, self.ccount = np.unique(dataset_y, return_counts=True)
        
        self.seq_length = seq_length
        
        # Samples of categories with less than this number of samples will be ignored
        self.drop_categos = []
        for cat, count in zip(self.categories, self.ccount):
            if count < drop_threshold:
                self.drop_categos.append(cat)
        
        # Remaining categories
        self.categories = np.setdiff1d(self.categories, self.drop_categos)
        
    def __iter__(self):
        for tweet, cat in zip(self.dataset_x, self.dataset_y):
            if cat in self.drop_categos:
                continue
            
            # Make all characters lower-case
            tweet = tweet.lower()
            
            # Clean string of all punctuation
            # tweet = tweet.translate(self.translator)
            tweet= preprocess(tweet)


            words = np.array([word2token(w) for w in tweet[:self.seq_length] if w != ''])
                                
            yield (words, cat)

sequences = SequenceIterator(x_train,y_train, DROP_THRESHOLD, MAX_SEQUENCE_LENGTH)

# Used for generating the labels in the set
cat_dict = {k: v for k, v in zip(sequences.categories, range(len(sequences.categories)))}

set_x = []
set_y = []
for w, c in sequences:
    set_x.append(w)
    set_y.append(cat_dict[c])
    
# Padding sequences with 0.
set_x = pad_sequences(set_x, maxlen=MAX_SEQUENCE_LENGTH, padding='pre', value=0)
set_y = np.array(set_y)

print(set_x.shape)
print(set_y.shape)

In [ ]:
# import random

# VALID_PER = 0.20 # Percentage of the whole set that will be separated for validation

# total_samples = set_x.shape[0]
# n_val = int(VALID_PER * total_samples)
# n_train = total_samples - n_val

# random_i = random.sample(range(total_samples), total_samples)
# train_x = set_x[random_i[:n_train]]
# train_y = set_y[random_i[:n_train]]
# val_x = set_x[random_i[n_train:n_train+n_val]]
# val_y = set_y[random_i[n_train:n_train+n_val]]

# # print(train_x)

# print("Train Shapes - X: {} - Y: {}".format(train_x.shape, train_y.shape))
# print("Val Shapes - X: {} - Y: {}".format(val_x.shape, val_y.shape))

# # Let's look at the distribution of categories in both sets
# categories, ccount = np.unique(train_y, return_counts=True)
# plt.figure(figsize=(16, 8))
# plt.title("Training Set - Category Distribution")
# plt.xticks(range(len(categories)), cat_dict.keys())
# plt.bar(categories, ccount, align='center')
# plt.show()

# categories, ccount = np.unique(val_y, return_counts=True)
# plt.figure(figsize=(16, 8))
# plt.title("Validation Set - Category Distribution")
# plt.xticks(range(len(categories)), cat_dict.keys())
# plt.bar(categories, ccount, align='center')
# plt.show()

# n_categories = len(categories)


In [ ]:
import multiprocessing

from gensim.models import Word2Vec
from time import time  # To time our operations

# cores = multiprocessing.cpu_count() # Count the number of cores in a computer

# w2v_model = Word2Vec(min_count=20,
#                      window=2,
#                      vector_size=100,
#                      sample=6e-5, 
#                      alpha=0.03, 
#                      min_alpha=0.001, 
#                      negative=20,
#                      workers=cores-3)
# t = time()

# w2v_model.build_vocab(sentences, progress_per=10000)

# print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

# # t = time()

# w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

# print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

# w2v_model.wv.most_similar(positive=["cowok"])
# print(data1)


In [ ]:
from keras.optimizers import RMSprop

# model = Sequential()

# # Keras Embedding layer with Word2Vec weights initialization
# model.add(Embedding(input_dim=vocab_size,
#                     output_dim=embedding_size,
#                     weights=[w2v_weights],
#                     input_length=MAX_SEQUENCE_LENGTH,
#                     mask_zero=True,
#                     trainable=False))

# model.add(LSTM(100))
# model.add(Dense(n_categories, activation='softmax'))

# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# history = model.fit(train_x, train_y, epochs=5, batch_size=64,
#                     validation_data=(val_x, val_y), verbose=1)

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=MAX_SEQUENCE_LENGTH))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100,dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint

stop = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=5
)

checkpoint= ModelCheckpoint(
    filepath='./',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
history=model.fit(set_x,set_y,batch_size=1024,epochs=50,
          validation_split=0.2,callbacks=[stop,checkpoint])

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.figure(figsize=(10, 10))
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Accuracy')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
sequences = SequenceIterator(x_test,y_test, DROP_THRESHOLD, MAX_SEQUENCE_LENGTH)

# Used for generating the labels in the set
cat_dict = {k: v for k, v in zip(sequences.categories, range(len(sequences.categories)))}

set_x_test = []
set_y_test = []
for w, c in sequences:
    set_x_test.append(w)
    set_y_test.append(cat_dict[c])
    
# Padding sequences with 0.
set_x_test = pad_sequences(set_x_test, maxlen=MAX_SEQUENCE_LENGTH, padding='pre', value=0)
set_y_test = np.array(set_y_test)

In [ ]:
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
MAX_SEQUENCE_LENGTH = 200

# tokenizer = Tokenizer(num_words=MAX_SEQUENCE_LENGTH)
# tokenizer.fit_on_texts(set_x_test)
# test_sequences = tokenizer.texts_to_sequences(set_x_test)
test_sequences_matrix = sequence.pad_sequences(set_x_test,maxlen=MAX_SEQUENCE_LENGTH)

accr = model.evaluate(test_sequences_matrix,y_test)

In [ ]:
# from keras.preprocessing.text import Tokenizer

# MAX_SEQUENCE_LENGTH = 200

# tokenizer = Tokenizer(num_words=MAX_SEQUENCE_LENGTH)
# tokenizer.fit_on_texts(x_test)
# test_sequences = tokenizer.texts_to_sequences(x_test)
# test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=MAX_SEQUENCE_LENGTH)

# accr = model.evaluate(test_sequences_matrix,y_test)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

lstm_prediction=model.predict(test_sequences_matrix)
res=[]
for prediction in lstm_prediction:
    if prediction[0]<0.5:
        res.append(0)
    else:
        res.append(1)
print(confusion_matrix(set_y_test,res))
print (classification_report(set_y_test, res))


In [ ]:
import pickle

with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(set_x, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
model.save("hate_model.h5")

In [ ]:
import keras
load_model=keras.models.load_model("./hate_model.h5")
with open('tokenizer.pickle', 'rb') as handle:
    load_tokenizer = pickle.load(handle)

In [ ]:
class TestIterator:
    def __init__(self, dataset):
        self.dataset = dataset

        # The text contains a stupid special character, hence the + '–'
        self.translator = str.maketrans('', '', string.punctuation + '–')

    def __iter__(self):
        for tweet in self.dataset:
            # Make all characters lower-case
            tweet = tweet.lower()
            tweet = preprocess(tweet)
            words = [word2token(w)
                             for w in tweet[:MAX_SEQUENCE_LENGTH] if w != '']

            # words = [w for w in sentence if w != '']
            # for sentence in tweet.split('.')[:-1]:
            #     # Clean string of all punctuation
            #     # sentence = sentence.translate(self.translator)
            #     sentence= preprocess(sentence)
            #     words = [w for w in sentence]

            yield words

In [ ]:
test = "- disaat semua cowok berusaha melacak perhatian gue. loe lantas remehkan perhatian yg gue kasih khusus ke elo. basic elo cowok bego ! ! !'"

sequences = SequenceIterator(test,y_test, DROP_THRESHOLD, MAX_SEQUENCE_LENGTH)

# Used for generating the labels in the set
cat_dict = {k: v for k, v in zip(sequences.categories, range(len(sequences.categories)))}

predict_x_test = []
set_y_test = []
for w, c in sequences:
    predict_x_test.append(w)
    set_y_test.append(cat_dict[c])
    
# Padding sequences with 0.
predict_x_test = pad_sequences(predict_x_test, maxlen=MAX_SEQUENCE_LENGTH, padding='pre', value=0)

In [ ]:


test=[preprocess(test)]
print(test)
seq = tokenizer.texts_to_sequences(test)
padded = sequence.pad_sequences(seq, maxlen=200)
print(seq)
pred = model.predict(padded)
print("pred", pred)
if pred<0.5:
    print("no hate")
else:
    print("hate")


In [ ]:
test = "- disaat semua cowok berusaha melacak perhatian gue. loe lantas remehkan perhatian yg gue kasih khusus ke elo. basic elo cowok bego ! ! !'"


test=[preprocess(test)]
# print(predict_x_test)
# seq = tokenizer.texts_to_sequences(test)
seq = TestIterator(test)
padded = pad_sequences(seq, maxlen=200)
pred = model.predict(padded)[0]
print("pred", pred)
if pred<0.5:
    print("no hate")
else:
    print("hate")

In [ ]:
lstm_prediction=model.predict(predict_x_test)
res=[]
for prediction in lstm_prediction:
    if prediction[0]<0.5:
        res.append(0)
    else:
        res.append(1)
print(res)